In [1]:
import os
import sys
if '/home/zechengh/Mastik/ad/detector/' not in sys.path:
    sys.path.append('/home/zechengh/Mastik/ad/detector/')
from collections import OrderedDict
    
import numpy as np
import torch
import matplotlib.pyplot as plt
%matplotlib inline

import utils
import ADbenchmark
import LSTMAD

import json
import collections

import torch

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

np.random.seed(0)
torch.manual_seed(0)

# Do not write .pyc
sys.dont_write_bytecode = True

# Reload code when code is changed
%load_ext autoreload
%autoreload 2

id_to_feature = utils.id_to_feature
for k, v in id_to_feature.items():
    print(k, v)

/home/zechengh/.local/lib/python3.6/site-packages/numba/errors.py:137: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


0 Ins
1 L1D read access (# load)
2 L1D read miss
3 L1D write access (# store)
4 L1D write miss
5 L1D prefetch miss
6 L1I read miss
7 LLC read access
8 LLC read miss
9 LLC write access
10 LLC write miss
11 LLC prefetch access
12 LLC prefetch miss
13 DTLB read access
14 DTLB read miss
15 DTLB write access
16 DTLB write miss
17 ITLB read access
18 ITLB read miss
19 BPU read access
20 BPU read miss
21 Cache node read access
22 Cache node read miss
23 Cache node write access
24 Cache node write miss
25 Cache node prefetch access
26 Cache node prefetch miss
27 cycles
28 branch instructions
29 branch prediction miss
30 page faults
31 context switch
32 stall_during_issue
33 stall_during_retirement
34 Time stamp


In [2]:
id_to_feature = utils.id_to_feature
data = collections.defaultdict(collections.defaultdict)

for bg_program in ['none', 'mysql', 'webserver', 'streamserver', 'mltrain', 'mapreduce']:
    data_dir = 'perf/data/{bg_program}_same_core/10000us/'.format(bg_program=bg_program)
    for f in os.listdir(data_dir):
        if f.endswith('.npy'):
            file_name = f.split('.')[0]
            data[bg_program][file_name] = np.load(os.path.join(data_dir, f))

feature_list = utils.FeatureSelect.feature_list

print("Used features:")
for i in feature_list:
    print(id_to_feature[i])

Used features:
Ins
stall_during_issue
stall_during_retirement
cycles
L1D read access (# load)
DTLB read access
L1D write access (# store)
BPU read access
DTLB write access
branch instructions
L1D read miss
L1I read miss
context switch


In [3]:
data_merged_train = np.concatenate(
    [
        data['mltrain']['train_normal'],
        data['mysql']['train_normal'],
        data['streamserver']['train_normal'],
        data['webserver']['train_normal'],
        data['mapreduce']['train_normal'],
    ],
    axis=0
)

data_merged_ref_and_val_normal = np.concatenate(
    [
        data['mltrain']['ref_and_val_normal'],
        data['mysql']['ref_and_val_normal'],
        data['streamserver']['ref_and_val_normal'],
        data['webserver']['ref_and_val_normal'],
        data['mapreduce']['ref_and_val_normal'],
    ],
    axis=0
)

data_merged_test = np.concatenate(
    [
        data['mltrain']['test_normal'],
        data['mysql']['test_normal'],
        data['streamserver']['test_normal'],
        data['webserver']['test_normal'],
        data['mapreduce']['test_normal']
    ],
    axis=0
)

print(data_merged_train.shape)
print(data_merged_ref_and_val_normal.shape)
print(data_merged_test.shape)

np.save('perf/data/merged/10000us/train_normal.npy', data_merged_train)
np.save('perf/data/merged/10000us/ref_and_val_normal.npy', data_merged_ref_and_val_normal)
np.save('perf/data/merged/10000us/test_normal.npy', data_merged_test)

(55000, 35)
(55000, 35)
(55000, 35)


python3 LSTMAD.py --training --data_dir ../perf/data/merged/10000us/ --save_model_name AnomalyDetectorMerged.ckpt --gpu --Nhidden 256 --LearningRate 1e-2 --Nbatches 100 --RED_points 20